In [ ]:
# bibliotecas necessárias
import pandas as pd
from google import colab as cl

# importando o arquivo para o
file_upload = cl.files.upload()

Saving train.csv to train.csv


In [ ]:
df = pd.read_csv( 'train.csv' )

In [ ]:
df1  = df.copy()

# 1. convertendo a coluna Age dde texto para numero
linhas_selecionas = (df1['Delivery_person_Age'] != 'NaN ')
df1 = df1.loc[linhas_selecionas, :].copy()

df1['Delivery_person_Age'] = df1['Delivery_person_Age'].astype(int)
df1.shape

# 2. convertendo a coluna Ratings de texto para numero decimal
df1['Delivery_person_Ratings'] = df1['Delivery_person_Ratings'].astype(float)

# 3. convertendo a coluna Order_Date de texto para data
df1['Order_Date'] = pd.to_datetime(df1['Order_Date'], format='%d-%m-%Y')

# 4. convertendo multiple_deliveries de texto para numero int
linhas_selecionas = (df1['multiple_deliveries'] != 'NaN ')
df1 = df1.loc[linhas_selecionas, :].copy()
df1['multiple_deliveries'] = df1['multiple_deliveries'].astype(int)

# 5. Removendo os epaços dentro de strings/testo/object


In [ ]:
df1.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,11:45:00,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,(min) 24
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45:00,19:50:00,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,(min) 33
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,08:45:00,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,(min) 26
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,18:10:00,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,(min) 21
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,13:45:00,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,(min) 30


# 1. Quantidade de pedidos por dia.

In [ ]:
# colunas
cols = ['ID', 'Order_Date']

# linhas
df_aux = df1.loc[:, cols].groupby('Order_Date').count().reset_index()

# desenhar o gráfico de linhas
# Plotly
import plotly.express as px

px.bar(df_aux, x='Order_Date', y='ID')

# 2. Quantidade de pedidos por semana.

In [ ]:
# criar a coluna de semana
df1['week_of_year'] = df1['Order_Date'].dt.strftime('%U')

df_aux = df1.loc[:, ['ID', 'week_of_year']].groupby('week_of_year').count().reset_index()

px.line(df_aux, x='week_of_year', y='ID')

# 3. Distribuição dos pedidos por tipo de tráfego.

In [ ]:
df_aux = df1.loc[:, ['ID', 'Road_traffic_density']].groupby('Road_traffic_density').count().reset_index()

df_aux = df_aux.loc[df_aux['Road_traffic_density'] != 'NaN', :]
df_aux['entregas_perc'] = df_aux['ID'] / df_aux['ID'].sum()

px.pie(df_aux, values='entregas_perc', names='Road_traffic_density')

# 4. Comparação do volume de pedidos por cidade e tipo de tráfego.

In [ ]:
df_aux = df1.loc[:, ['ID', 'City', 'Road_traffic_density']].groupby(['City', 'Road_traffic_density']).count().reset_index()
df_aux = df_aux.loc[df_aux['City'] != 'NaN', :]
df_aux = df_aux.loc[df_aux['Road_traffic_density'] != 'NaN', :]

px.scatter(df_aux, x= 'City', y='Road_traffic_density', size='ID', color='City')

# 5. A quantidade de pedidos por entregador por semana.

In [ ]:
# Qtd de pedidos por semana / Número de entregadores por semana
df_aux1 = df1.loc[:, ['ID', 'week_of_year']].groupby('week_of_year').count().reset_index()
df_aux2 = df1.loc[:, ['Delivery_person_ID', 'week_of_year']].groupby('week_of_year').nunique().reset_index()

df_aux = pd.merge(df_aux1, df_aux2, how='inner')

df_aux['order_by_delivery'] = df_aux['ID'] / df_aux['Delivery_person_ID']

px.line(df_aux, x='week_of_year', y='order_by_delivery')


# 6. A localização central de cada cidade por tipo de tráfego.

In [ ]:
df_aux = df.loc[:, ['City', 'Road_traffic_density', 'Delivery_location_latitude', 'Delivery_location_longitude']]. groupby(['City', 'Road_traffic_density']).median().reset_index()

df_aux = df_aux.loc[df_aux['City'] != 'NaN', :]
df_aux = df_aux.loc[df_aux['Road_traffic_density'] != 'NaN', :]

import folium
map = folium.Map()

for index, location_info in df_aux.iterrows():
  folium.Marker( [location_info['Delivery_location_latitude'],
                 location_info['Delivery_location_longitude']],
                 popup=location_info[['City', 'Road_traffic_density']]).add_to(map)

map